# Code-Beispiel: Regularisierung+CV

In diesem Notebook wollen wir die logitische Regression aus Notebook 2 erweitern, indem wir Regularisierung einführen und mit Hilfe von Cross Validation den besten Parameter dafür finden.

Hinweis: Dieses Notebook folgt im großen Teilen dieser [Quelle](http://www.science.smith.edu/~jcrouser/SDS293/labs/lab10-py.html).

### Daten laden und vorbereiten

Wir laden den `Hitters`-Datensatz, bei dem es darum geht anhand von Attributen von Baseball-Spielern deren Gehalt ("`Salary`") vorherzusagen.

In [29]:
import pandas as pd
df = pd.read_csv('Hitters.txt')
df.head().T

,0,1,2,3,4
Player,-Andy Allanson,-Alan Ashby,-Alvin Davis,-Andre Dawson,-Andres Galarraga
AtBat,293,315,479,496,321
Hits,66,81,130,141,87
HmRun,1,7,18,20,10
Runs,30,24,66,65,39
RBI,29,38,72,78,42
Walks,14,39,76,37,30
Years,1,14,3,11,2
CAtBat,293,3449,1624,5628,396
CHits,66,835,457,1575,101


Im ersten Schritt entfernen wir alle Werte mit N/A Werten und entfernen den Namen, welcher für kein Feature verwendet wird.

In [30]:
df = df.dropna().drop('Player', axis = 1)

Die Daten enthalten drei kategorische Attribute, welche one-hot-encoded werden:

In [36]:
dummies = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
dummies

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0
...,...,...,...,...,...,...
317,0,1,1,0,0,1
318,1,0,1,0,1,0
319,1,0,0,1,1,0
320,1,0,1,0,1,0


Im nächsten Schritt definieren wir das Label und die Feature. Hierbei selektieren wir nur ein Teil der one-hot-encoded Features.

In [39]:
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis = 1)
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis = 1)

X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,1,1,1
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,0,1,0
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,1,0,1
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,1,0,1
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,0,1,0


In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

### Lineare Regression

In [52]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(normalize = True) # normalisiert automatisch die Daten
reg.fit(X_train, y_train)

LinearRegression(normalize=True)

Als nächstes schauen wir auf den MSE auf den Testdaten und die einzelnen Featuregewichte:

In [53]:
pred = reg.predict(X_test)      
print(pd.Series(reg.coef_, index = X.columns)) 
print(mean_squared_error(y_test, pred))        

AtBat           -1.821115
Hits             4.259156
HmRun           -4.773401
Runs            -0.038760
RBI              3.984578
Walks            3.470126
Years            9.498236
CAtBat          -0.605129
CHits            2.174979
CHmRun           2.979306
CRuns            0.266356
CRBI            -0.598456
CWalks           0.171383
PutOuts          0.421063
Assists          0.464379
Errors          -6.024576
League_N       133.743163
Division_W    -113.743875
NewLeague_N    -81.927763
dtype: float64
116690.46856661116


### Ridge Regression

Wir trainieren nun eine Ridge-Regression, um eine Regularisierung zu verwenden. Dazu probieren wir zuerst den Parameter alpha = 4 als Gewichtung für den Regressionsterm aus.

In [46]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 4, normalize = True)
ridge.fit(X_train, y_train)

Ridge(alpha=4, normalize=True)

Als nächstes schauen wir auf den MSE auf den Testdaten und die einzelnen Featuregewichte:

In [47]:
pred = ridge.predict(X_test)           # Use this model to predict the test data
print(pd.Series(ridge.coef_, index = X.columns)) # Print coefficients
print(mean_squared_error(y_test, pred))          # Calculate the test MSE

AtBat           0.098658
Hits            0.446094
HmRun           1.412107
Runs            0.660773
RBI             0.843403
Walks           1.008473
Years           2.779882
CAtBat          0.008244
CHits           0.034149
CHmRun          0.268634
CRuns           0.070407
CRBI            0.070060
CWalks          0.082795
PutOuts         0.104747
Assists        -0.003739
Errors          0.268363
League_N        4.241051
Division_W    -30.768885
NewLeague_N     4.123474
dtype: float64
106216.52238005561


Wir sehen dass die Gewichte im Schnitt kleiner sind und dass wir den Test-MSE verringern konnten.

Testweise probieren wir ein hohes alpha = 10^10 aus:

In [48]:
ridge3 = Ridge(alpha = 10**10, normalize = True)
ridge3.fit(X_train, y_train)             # Fit a ridge regression on the training data
pred3 = ridge3.predict(X_test)           # Use this model to predict the test data
print(pd.Series(ridge3.coef_, index = X.columns)) # Print coefficients
print(mean_squared_error(y_test, pred3))          # Calculate the test MSE

AtBat          1.317464e-10
Hits           4.647486e-10
HmRun          2.079865e-09
Runs           7.726175e-10
RBI            9.390640e-10
Walks          9.769219e-10
Years          3.961442e-09
CAtBat         1.060533e-11
CHits          3.993605e-11
CHmRun         2.959428e-10
CRuns          8.245247e-11
CRBI           7.795451e-11
CWalks         9.894387e-11
PutOuts        7.268991e-11
Assists       -2.615885e-12
Errors         2.084514e-10
League_N      -2.501281e-09
Division_W    -1.549951e-08
NewLeague_N   -2.023196e-09
dtype: float64
172862.23580379886


Wir sehen dass die Gewichte jetzt alle auf nahezu Null gedrückt werden, der MSE aber dadurch riesig wird.

Mehr bla bla an dieser Stelle

In [58]:
import numpy as np
alphas = 10**np.linspace(10,-2,100)*0.5

In [59]:
from sklearn.linear_model import RidgeCV
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)
ridgecv.alpha_

0.5748784976988678

In [60]:
ridge4 = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridge4.fit(X_train, y_train)
mean_squared_error(y_test, ridge4.predict(X_test))

99825.6489629273

### Lasso

In [63]:
from sklearn.linear_model import Lasso, LassoCV

lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)

lasso = Lasso(max_iter = 10000, normalize = True)
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
mean_squared_error(y_test, lasso.predict(X_test))

104960.65853895502

In [64]:
pd.Series(lasso.coef_, index=X.columns)

AtBat           0.000000
Hits            1.082446
HmRun           0.000000
Runs            0.000000
RBI             0.000000
Walks           2.906388
Years           0.000000
CAtBat          0.000000
CHits           0.000000
CHmRun          0.219367
CRuns           0.000000
CRBI            0.513975
CWalks          0.000000
PutOuts         0.368401
Assists        -0.000000
Errors         -0.000000
League_N        0.000000
Division_W    -89.064338
NewLeague_N     0.000000
dtype: float64